In [2]:
# %%
from pathlib import Path 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
from napatrackmater import  create_analysis_tracklets, convert_tracks_to_simple_arrays
from napatrackmater.Trackvector import (TrackVector,
                                        create_cluster_plot,
                                        SHAPE_FEATURES, 
                                        DYNAMIC_FEATURES, 
                                        SHAPE_DYNAMIC_FEATURES,
                                        
                                        )

2024-07-15 14:59:31.598350: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 14:59:31.677998: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 14:59:32.962632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/debian/miniconda3/envs/kapoorlabsenv/lib/python3.10/site-packages/pyntcloud/utils/numba.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-beha

In [3]:
dataset_name = 'Fourth'
home_folder = '/home/debian/jz/'
timelapse_to_track = f'timelapse_{dataset_name.lower()}_dataset'
tracking_directory = f'{home_folder}/Mari_Data_Oneat/Mari_{dataset_name}_Dataset_Analysis/nuclei_vollseg_membrane_tracking/'
channel = 'nuclei_vollseg_'
data_frames_dir = os.path.join(tracking_directory, f'dataframes/')
master_xml_name = 'master_' + 'marching_cubes_filled_' + channel + timelapse_to_track + ".xml"
xml_path = Path(os.path.join(tracking_directory, master_xml_name))
     
time_delta = 10

normalized_dataframe = os.path.join(data_frames_dir , f'results_dataframe_normalized_{channel}.csv')

save_dir = os.path.join(tracking_directory, f'hiearchial_clustering_plots/')
Path(save_dir).mkdir(exist_ok=True, parents=True)
method="ward"
criterion="distance"
metric="euclidean" 

shape_cols = SHAPE_FEATURES
dynamic_cols = DYNAMIC_FEATURES
feature_cols = SHAPE_DYNAMIC_FEATURES



cluster_threshold_shape_dynamic_range=[20 + i for i in range(1, 80)]
cluster_threshold_dynamic_range = cluster_threshold_shape_dynamic_range
cluster_threshold_shape_range = cluster_threshold_shape_dynamic_range


In [ ]:



track_vectors = TrackVector(master_xml_path=xml_path)
track_vectors.t_minus = 0
track_vectors.t_plus = track_vectors.tend
track_vectors.y_start = 0
track_vectors.y_end = track_vectors.ymax
track_vectors.x_start = 0
track_vectors.x_end = track_vectors.xmax

print(f'reading data from {normalized_dataframe}')
tracks_dataframe = pd.read_csv(normalized_dataframe)

tracks_dataframe['Cell_Type'] = 1


total_dividing_tracklets = tracks_dataframe[tracks_dataframe['Dividing'] == 1]['Track ID'].nunique()

total_non_dividing_tracklets = tracks_dataframe[tracks_dataframe['Dividing'] == 0]['Track ID'].nunique()

print(f'Total Tracklets dividing tracks {total_dividing_tracklets}, non-dividing {total_non_dividing_tracklets}')


total_dividing_tracklets = tracks_dataframe[tracks_dataframe['Dividing'] == 1]['TrackMate Track ID'].nunique()

total_non_dividing_tracklets = tracks_dataframe[tracks_dataframe['Dividing'] == 0]['TrackMate Track ID'].nunique()

print(f'Total TrackMate dividing tracks {total_dividing_tracklets}, non-dividing {total_non_dividing_tracklets}')



In [ ]:
t_start = 0
t_delta = time_delta
t_total = max(tracks_dataframe['t'])
starting_label_shape_dynamic = 0
starting_label_dynamic = 0
starting_label_shape = 0

copy_dataframe = tracks_dataframe.copy()



local_dataframe_dict = {}
save_file_shape_dynamic = os.path.join(data_frames_dir , f'results_dataframe_shape_dynamic_sillhouette_normalized_{channel}delta_{t_delta}.csv')
save_file_dynamic = os.path.join(data_frames_dir , f'results_dataframe_dynamic_sillhouette_normalized_{channel}delta_{t_delta}.csv')
save_file_shape = os.path.join(data_frames_dir , f'results_dataframe_shape_sillhouette_normalized_{channel}delta_{t_delta}.csv') 


if not os.path.exists(save_file_shape_dynamic):
   
        for i in tqdm(np.arange(t_start, t_total, t_delta)):
                local_analysis_vectors, local_shape_dynamic_dataframe = create_analysis_tracklets(copy_dataframe, t_minus=i, t_plus=i + t_delta) 

               

                result =  convert_tracks_to_simple_arrays(local_analysis_vectors, metric=metric,
                                    cluster_threshold_shape_dynamic=cluster_threshold_shape_dynamic_range,
        cluster_threshold_dynamic=cluster_threshold_dynamic_range,
        cluster_threshold_shape=cluster_threshold_shape_range,
        t_delta = t_delta,
            method=method,
            criterion=criterion)
                
               
                if result is not None:
                        ( local_shape_dynamic_covariance_2d,
                        local_shape_covariance_2d,
                        local_dynamic_covariance_2d, 
                        local_shape_dynamic_cluster_labels_dict, 
                        local_shape_cluster_labels_dict, 
                        local_dynamic_cluster_labels_dict,
                        local_shape_dynamic_linkage_matrix, 
                        local_shape_linkage_matrix, 
                        local_dynamic_linkage_matrix, 
                        local_shape_dynamic_silhouette,
                        local_shape_dynamic_wcss_value,
                        local_shape_silhouette,
                        local_shape_wcss_value,
                        local_dynamic_silhouette,
                        local_dynamic_wcss_value,
                        local_cluster_distance_map_shape_dynamic_dict,
                        local_cluster_distance_map_shape_dict,
                        local_cluster_distance_map_dynamic_dict,
                        local_cluster_eucledian_distance_map_shape_dynamic_dict,
                        local_cluster_eucledian_distance_map_shape_dict,
                        local_cluster_eucledian_distance_map_dynamic_dict,
                        local_analysis_track_ids) = result
                        local_shape_dynamic_dataframe = local_shape_dynamic_dataframe.copy()
                        tracks_dataframe.loc[:, f"Shape Dynamic Cluster_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_shape_dynamic_cluster_labels_dict)
                        tracks_dataframe.loc[:, f"Shape Cluster_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_shape_cluster_labels_dict)
                        tracks_dataframe.loc[:, f"Dynamic Cluster_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_dynamic_cluster_labels_dict)
                        tracks_dataframe.loc[:, f"Shape Dynamic Intra Cluster Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_distance_map_shape_dynamic_dict)
                        tracks_dataframe.loc[:, f"Shape Intra Cluster Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_distance_map_shape_dict)
                        tracks_dataframe.loc[:, f"Dynamic Intra Cluster Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_distance_map_dynamic_dict)
                        tracks_dataframe.loc[:, f"Shape Dynamic Intra Cluster Eucledian Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_eucledian_distance_map_shape_dynamic_dict)
                        tracks_dataframe.loc[:, f"Shape Intra Cluster Eucledian Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_eucledian_distance_map_shape_dict)
                        tracks_dataframe.loc[:, f"Dynamic Intra Cluster Eucledian Distance_{i}"] = local_shape_dynamic_dataframe["Track ID"].map(local_cluster_eucledian_distance_map_dynamic_dict)
                        

                       

      



In [ ]:


        
if not os.path.exists(save_file_dynamic):
    cluster_dynamic_dataframe = create_cluster_plot(dataframe=tracks_dataframe, cluster_type='Dynamic Cluster', cluster_distance_type='Dynamic Intra Cluster Distance',cluster_eucledian_distance_type='Dynamic Intra Cluster Eucledian Distance', negate_cluster_type='Shape Dynamic Cluster', negate_cluster_distance_type='Shape Dynamic Intra Cluster Distance',negate_cluster_eucledian_distance_type='Shape Dynamic Intra Cluster Eucledian Distance')
    cluster_shape_dynamic_dataframe = create_cluster_plot(dataframe=tracks_dataframe, cluster_type='Shape Dynamic Cluster', cluster_distance_type= 'Shape Dynamic Intra Cluster Distance', cluster_eucledian_distance_type= 'Shape Dynamic Intra Cluster Eucledian Distance')
    cluster_shape_dataframe = create_cluster_plot(dataframe=tracks_dataframe, cluster_type='Shape Cluster', negate_cluster_type='Shape Dynamic Cluster',cluster_distance_type='Shape Intra Cluster Distance', cluster_eucledian_distance_type='Shape Intra Cluster Eucledian Distance', negate_cluster_distance_type='Shape Dynamic Intra Cluster Distance', negate_cluster_eucledian_distance_type='Shape Dynamic Intra Cluster Eucledian Distance')
    
    cluster_dynamic_dataframe.to_csv(save_file_dynamic)
    cluster_shape_dynamic_dataframe.to_csv(save_file_shape_dynamic)
    cluster_shape_dataframe.to_csv(save_file_shape)
else:
    cluster_dynamic_dataframe = pd.read_csv(save_file_dynamic)
    cluster_shape_dynamic_dataframe = pd.read_csv(save_file_shape_dynamic)
    cluster_shape_dataframe = pd.read_csv(save_file_shape)         


In [ ]:
hue_options = ["Dynamic_Cluster_Label_Distances", "Dynamic_Cluster_Label_Eucledian_Distances"]
cluster_plots = DYNAMIC_FEATURES
dynamic_save_dir = os.path.join(save_dir, 'Dynamic_Clustering/')
Path(dynamic_save_dir).mkdir(exist_ok=True, parents=True)
for cluster_plot in cluster_plots:
            filtered_tracks = cluster_dynamic_dataframe[cluster_dynamic_dataframe['Track Duration'] >= 0]
            
            for hue_option in hue_options:                  
                plt.figure(figsize=(15, 6))
                scatter = plt.scatter(filtered_tracks['t'],filtered_tracks[cluster_plot],c=filtered_tracks[hue_option],cmap='viridis')
                plt.xlabel('Time (t)')
                plt.ylabel(f'{cluster_plot}')
            
                plt.colorbar(scatter, label = hue_option)
                plt.title(f'Dynamic feature based clustering')
                fig_name = f"{channel}{cluster_plot}_{hue_option}_all.png"
                plt.savefig(os.path.join(dynamic_save_dir, fig_name))
                plt.show()

for count, hue_option in enumerate(hue_options):    
                unique_t_values = cluster_dynamic_dataframe['t'].unique()     
                t_values = []
                mean_update_column_values = []     
                for t_value in unique_t_values:
                    filtered_data_t = cluster_dynamic_dataframe[cluster_dynamic_dataframe['t'] == t_value]
                    mean_update_column = filtered_data_t[hue_option].mean()
                    t_values.append(t_value)
                    mean_update_column_values.append(mean_update_column)    
                plt.figure(figsize=(15, 6))
                plt.plot(t_values, mean_update_column_values, marker='o')  
                plt.xlabel('Time (t)')
                plt.ylabel('Shape feature distance')
                plt.title(f'Dynamic feature {hue_option}')
                fig_name = f"{channel}{hue_option}_line_plot_all.png"
                plt.savefig(os.path.join(dynamic_save_dir, fig_name))
                plt.show()  

hue_options = ["Shape_Cluster_Label_Distances", "Shape_Cluster_Label_Eucledian_Distances"]
cluster_plots = SHAPE_FEATURES
shape_save_dir = os.path.join(save_dir, 'Shape_Clustering/')
Path(shape_save_dir).mkdir(exist_ok=True, parents=True)
for cluster_plot in cluster_plots:
            filtered_tracks = cluster_shape_dataframe[cluster_shape_dataframe['Track Duration'] >= 0]
           
            for hue_option in hue_options:                 
                plt.figure(figsize=(15, 6))
                scatter = plt.scatter(filtered_tracks['t'],filtered_tracks[cluster_plot],c=filtered_tracks[hue_option],cmap='viridis')
                plt.xlabel('Time (t)')
                plt.ylabel(f'{cluster_plot}')
            
                plt.colorbar(scatter, label = hue_option)
                plt.title(f'Shape feature based clustering')
                fig_name = f"{channel}{cluster_plot}_{hue_option}_all.png"
                plt.savefig(os.path.join(shape_save_dir, fig_name))
                plt.show()

for count, hue_option in enumerate(hue_options):    
                unique_t_values = cluster_shape_dataframe['t'].unique()     
                t_values = []
                mean_update_column_values = []     
                for t_value in unique_t_values:
                    filtered_data_t = cluster_shape_dataframe[cluster_shape_dataframe['t'] == t_value]
                    mean_update_column = filtered_data_t[hue_option].mean()
                    t_values.append(t_value)
                    mean_update_column_values.append(mean_update_column)    
                plt.figure(figsize=(15, 6))
                plt.plot(t_values, mean_update_column_values, marker='o')  
                plt.xlabel('Time (t)')
                plt.ylabel('Dynamic feature distance')
                plt.title(f'Shape feature {hue_option}')
                fig_name = f"{channel}{hue_option}_line_plot_all.png"
                plt.savefig(os.path.join(shape_save_dir, fig_name))
                plt.show() 
                                 